In [ ]:
import torch
import numpy as np
import math

import pykeops
#pykeops.clean_pykeops()

In [ ]:
from gechebnet.graph.graph import SE2GEGraph, SO3GEGraph
from gechebnet.model.chebnet import ResGEChebNet

In [ ]:
se2_graph = SE2GEGraph(
    nx=10,
    ny=10,
    ntheta=6,
    knn=16,
    sigmas=(xi / eps, xi, 1.0),
    weight_kernel=lambda sqdistc, sigmac: torch.exp(-sqdistc / sigmac),
    kappa=0.,
    device=torch.device("cuda")
)

In [ ]:
device = torch.device("cuda")

model = ResGEChebNet(se2_graph, 6, 3, 10, 5, device=device)
model = model.to(device)

In [ ]:
x = torch.rand(8, 3, 600, device=device)

In [ ]:
pred = model(x)